In [ ]:
#Xplane menu and initialization
#XP functions to be in python plugin folder
from XPPython3 import xp
class PythonInterface:
    def XPluginStart(self):
        self.StateDataRefDescriptions = ["sim/flightmodel/engine/ENGN_thro", "sim/joystick/yolk_pitch_ratio",
                                   "sim/joystick/yoke_roll_ratio","sim/joystick/FC_ptch",
                                   "sim/cockpit2/gauges/indicators/altitude_ft_pilot", "sim/flightmodel/position/true_psi",
                                   "sim/cockpit2/gauges/indicators/airspeed_kts_pilot"]
        self.ActionDataRefDescriptions = ["sim/flightmodel/engine/ENGN_thro", "sim/cockpit2/controls/total_pitch_ratio",
                                   "sim/cockpit2/controls/total_roll_ratio","sim/flightmodel/controls/rudd_def"]
        self.StateDataRef = []
        for Item in range(gym.obs_size):
            gym.InputDataRef.append(xp.findDataRef(StateDataRefDescriptions[Item]))
    
        self.ActionDataRef = []
        for Item in range(gym.action_size):
            gym.ActionDataRef.append(xp.findDataRef(ActionDataRefDescriptions[Item]))
      
        #XP init:  make item list
        Item = xp.appendMenuItem(xp.findPluginsMenu(), "Python - Position 1", 0)
        self.PositionMenuHandlerCB = self.PositionMenuHandler
        self.Id = xp.createMenu("Position1", xp.findPluginsMenu(), Item, self.PositionMenuHandlerCB, 0)
        xp.appendMenuItem(self.Id, "Position1", 1)
    
        # Flag to tell us if the widget is being displayed.
        self.MenuItem1 = 0
        return gym

    def XPluginStop(self):
        # Unregister the callback
        xp.unregisterFlightLoopCallback(self.InputOutputLoopCB, 0)

        if self.MenuItem1 == 1:
            xp.destroyWidget(self.InputOutputWidget, 1)
            self.MenuItem1 = 0

        xp.destroyMenu(self.Id)

    def XPluginEnable(self):
        return 1

    def XPluginDisable(self):
        pass

    def XPluginReceiveMessage(self, inFromWho, inMessage, inParam):
        pass
    
    def XPobs(self):
        Pitch=[]
        Roll = []
        #Yaw = []
        Throttle =[]
        Alt = []
        Hdg = []
        Speed = []
        count = xp.getDatavf(self.InputDataRef[0], Throttle, 0, 1)
    
        # #Get pitch, roll, yaw
    
        Pitch = xp.getDataf(self.InputDataRef[1])#, self.Pitch, 0, 1) #normalized
        Roll = xp.getDataf(self.InputDataRef[2])#, self.Roll, 0, 1)
        #Yaw = xp.getDataf(self.InputDataRef[3])#, self.Yaw, 0, 1)
    
        # #Get altitude, heading, airspeed
    
        Alt = xp.getDataf(self.InputDataRef[4])#, self.Alt, 0, 1)
        Hdg = xp.getDataf(self.InputDataRef[5])#, self.Hdg, 0, 1)
        Speed = xp.getDataf(self.InputDataRef[6])#, self.Speed, 0, 1)
        #convert to degrees and normalize to +/- 1 with guessed limits 
        SpeedNorm= (Speed-120.)/(170-70) #if speed outside these limits, then fail
        AltNorm= (Alt-4000)/(5500-2500) #if alt outside limits, fail
        HdgNorm = (Hdg-180)/(270-90) # hdg limis 90-270
        #all other already normalized
        return(Pitch, Roll, Throttle, SpeedNorm, AltNorm, HdgNorm)
    
    def XPreset():
        xp.registerFlightLoopCallback(self.InputOutputLoopCB, 3.0, 0)
        AltRef=xp.findDataRef("sim/cockpit2/gauges/indicators/altitude_ft_pilot")
        SpeedRef=xp.findDataRef("sim/cockpit2/gauges/indicators/airspeed_kts_pilot")
        HdgRef=xp.findDataRef("sim/flightmodel/position/true_psi")
        
        xp.setDataf(AltRef, 4000.)
        xp.setDataf(SpeedRef, 120.)
        xp.setDataf(HdgRef, 180.)
    
        # #send command
        # ThrottleCmd=xp.findDataRef("sim/flightmodel/engine/ENGN_thro")
        # PitchCmd=xp.findDataRef("sim/cockpit2/controls/total_pitch_ratio")
        # RollCmd=xp.findDataRef( "sim/cockpit2/controls/total_roll_ratio")
        
        # xp.setDataf(ThrottleCmd, action[0])
        # xp.setDataf(PitchCmd, action[1])
        # xp.setDataf(RollCmd, action[2])
       

    def XPaction(self,action):
        #send command
        ThrottleCmd=xp.findDataRef("sim/flightmodel/engine/ENGN_thro")
        PitchCmd=xp.findDataRef("sim/cockpit2/controls/total_pitch_ratio")
        RollCmd=xp.findDataRef( "sim/cockpit2/controls/total_roll_ratio")
        
        xp.setDataf(ThrottleCmd, action[0])
        xp.setDataf(PitchCmd, action[1])
        xp.setDataf(RollCmd, action[2])

        
        #wait X=3 seconds here, then get observation and reward
        xp.registerFlightLoopCallback(MyCallback, 3.0, 0)
        xp.unregisterFlightLoopCallback(MyCallback, 0)

    def MyCallback(lastCall, elapsedTime, counter, refCon):
    
       #xp.log(f"{elapsedTime}, {counter}")
    
       return 1.0
    
    def PositionMenuHandler(self, inMenuRef, inItemRef):
        # If menu selected create our widget dialog
        if inItemRef == 1:
            if self.MenuItem1 == 0:
                self.CreatePosition(300, 600, 300, 550)
                self.MenuItem1 = 1
            else:
                if not xp.isWidgetVisible(self.PositionWidget):
                    xp.showWidget(self.PositionWidget)
    
    def CreatePosition(self, x, y, w, h):
        FloatValue = []
        for Item in range(self.MAX_ITEMS):
            FloatValue.append(xp.getDataf(self.PositionDataRef[Item]))
    
        # X, Y, Z, Lat, Lon, Alt
        DoubleValue = [0.0, 0.0, 0.0]
        DoubleValue[0], DoubleValue[1], DoubleValue[2] = xp.localToWorld(FloatValue[0], FloatValue[1], FloatValue[2])
        DoubleValue[2] *= 3.28
    
        x2 = x + w
        y2 = y - h
        PositionText = []
    
        # Create the Main Widget window
        self.PositionWidget = xp.createWidget(x, y, x2, y2, 1, "Python - Position Example 1 by Sandy Barbour", 1,
                                              0, xp.WidgetClass_MainWindow)
    
        # Add Close Box decorations to the Main Widget
        xp.setWidgetProperty(self.PositionWidget, xp.Property_MainWindowHasCloseBoxes, 1)


In [1]:
#gymanisum
import gymnasium as gym
from gymnasium import spaces
from gymnasium.envs.classic_control import utils
from gymnasium.error import DependencyNotInstalled
##At the top of the code
import logging
logger = logging.getLogger('requests_throttler')
logger.addHandler(logging.NullHandler())
logger.propagate = False


#not sure where to put these

#diffusion equation is included in py-pde, so don't need seperate dynamics
#import dynamics
#from dynamics import MGM

class XPlaneEnv(gym.Env):
    #observation space is state and control spaces, normalized
    #space is 7-vector
    #also make variables to support XP python calls
    #also add menu selection slot for XP python
    def __init__(self, render_mode=None, size: int =7):
        #gymansium init:  make spaces
        obs_size=6
        self.observation_space =spaces.Box(low=-1, high=1, shape=(obs_size+2,), dtype=float) #add two for commands
        action_size=3
        self.action_space = spaces.Box(low=-1, high=1, shape=(action_size,), dtype=float) 
        #need to update action to normal distribution

        self.grid=[]
        self.stepper=[]
        #XP init:  make variable list
        # self.StateDataRefDescriptions = ["sim/flightmodel/engine/ENGN_thro", "sim/joystick/yolk_pitch_ratio",
        #                            "sim/joystick/yoke_roll_ratio","sim/joystick/FC_ptch",
        #                            "sim/cockpit2/gauges/indicators/altitude_ft_pilot", "sim/flightmodel/position/true_psi",
        #                            "sim/cockpit2/gauges/indicators/airspeed_kts_pilot"]
        # self.ActionDataRefDescriptions = ["sim/flightmodel/engine/ENGN_thro", "sim/cockpit2/controls/total_pitch_ratio",
        #                            "sim/cockpit2/controls/total_roll_ratio","sim/flightmodel/controls/rudd_def"]


        #XP call
        XPinit(self)
        
        
    def _get_obs(self, Obs):

        Pitch, Roll, Throttle, AltNorm, HdgNorm, SpeedNorm=Obs
        self.state.data=[Pitch, Roll, Throttle, AltNorm, HdgNorm, SpeedNorm, self.command[0], self.command[1]]
        return self.state

    def reset(self, seed: Optional[int] = None, options=None):
        #reset to altitude 4000, airspeed 120, heading 180
        #only doing relative heading/altitude changes so shouldn't affect things much
        # AltRef=xp.findDataRef("sim/cockpit2/gauges/indicators/altitude_ft_pilot")
        # SpeedRef=xp.findDataRef("sim/cockpit2/gauges/indicators/airspeed_kts_pilot")
        # HdgRef=xp.findDataRef("sim/flightmodel/position/true_psi")
        
        xp.setDataf(AltRef, 4000.)
        xp.setDataf(SpeedRef, 120.)
        xp.setDataf(HdgRef, 180.)

        #commands randmoly chosen from a matrix of possible commands 
        #+/- 45 deg heading, with fixed initial heading this is 225 or 135
        #+/- 500 ft elevation, with fixed initial altitude this is 4500 or 3500
        #combination of the two
        #commands is 2x8 array with heading change, elevation change as each row
        #commands = np.array([[225., 4000.], [135., 4000.], [180., 4500.], [180., 3500.], [225., 4500.], [135., 4500.], [225., 3500.], [135., 3500.]])
        #issue normalized commands
        commands = np.array([[45./180., 0.], [-45./180., 0.], [0., 500./3000], [0., -500./3000], [45./180, 500./3000], 
                             [-45./180, 500./3000], [45./180, -500./3000], [-45./180, -500./3000]])
        #choose a random command to send
        n=random.randint(0,7)
        self.command = commands[n,]

        observation = self._get_obs()
        
        return observation
        
    def step(self,action):
        #design of the training will need to be period of time (30s-1min sim time) where aagent tries to acheive command, then reset position and command
        #during each period T agent can continuously control pitch, roll, throttle (leave rudder/yaw out for now and just do uncoordinated)
        #if altitude change by more than 1500 feet or heading change by more than 90 degrees, or speed change by more than 50kts then fail
        #want to sample for reward and updating control input every X sim seconds (start with T/10)
        #what/how does step work then, is it every 6 seconds or an entire period?  Each step should be a period or a full contorl/reward interval
        #action to be a numpy 3-vector pitch commmand, roll command, throttle command
          
        #self.state.data[0,:]=action[0]
        state=self.state   
        XPaction(action)
        observation=self._get_obs()
        reward=np.sqrt((self.state.data[3]-self.command[1])**2+(self.state.data[4]-self.command[0])**2+(self.state.data[5]-120.)**2)
        done=False
        truncated=False
        if (self.state.data[3]>5500 or self.state.data[3]<2500 or self.state.data[4]>270 or self.state.data[4]<090 or 
            self.state.data[5]>170 or self.state.data[5]<70 ):
            truncated= True
            reward=-5000
        state=self.state
 
        return self.state, reward, done, truncated, {}
    
    
    def close(self):
        XPluginStop

In [5]:
#ppo setup

In [7]:
#ppo init

In [ ]:
#ppo run